#### Testing USDA API connection

In [2]:
import pandas as pd
import requests

In [ ]:
def fetch_yield(api_key:str, crop:str, start_year:str, end_year:str):
    """Fetches annual crop yield at the state level from USDA QuickStats."""

    base_url = "https://quickstats.nass.usda.gov/api/api_GET/"

    params = {
        "key": api_key,
        "commodity_desc": crop.upper(),
        "statisticcat_desc": "YIELD",
        "sector_desc": "CROPS",
        "source_desc": "SURVEY",
        "agg_level_desc": "STATE",
        "unit_desc": "BU / ACRE",
        "year__GE": start_year,
        "year__LE": end_year,
        "format": "JSON"
    }
    response = requests.get(url=base_url, params=params)
    response.raise_for_status()

    # print(f"Requesting USDA NASS API:\n{response.url}\n")
    return response.json()

In [8]:
from dotenv import load_dotenv
import os 

load_dotenv()
API_KEY = os.getenv("API_KEY")

data = fetch_yield(api_key=API_KEY, crop="corn", start_year="2021", end_year="2024")

Requesting USDA NASS API:
https://quickstats.nass.usda.gov/api/api_GET/?key=01ED94F8-A939-3783-BED5-A806A6C6C48B&commodity_desc=CORN&statisticcat_desc=YIELD&sector_desc=CROPS&source_desc=SURVEY&agg_level_desc=STATE&unit_desc=BU+%2F+ACRE&year__GE=2021&year__LE=2024&format=JSON



In [5]:
# pretty view
import json
print(json.dumps(data, indent=4))

{
    "data": [
        {
            "watershed_desc": "",
            "begin_code": "00",
            "country_name": "UNITED STATES",
            "state_fips_code": "98",
            "statisticcat_desc": "YIELD",
            "commodity_desc": "CORN",
            "source_desc": "SURVEY",
            "county_code": "",
            "week_ending": "",
            "state_alpha": "OT",
            "Value": "157.8",
            "CV (%)": "",
            "asd_code": "",
            "asd_desc": "",
            "state_ansi": "",
            "county_ansi": "",
            "congr_district_code": "",
            "watershed_code": "00000000",
            "year": 2025,
            "short_desc": "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",
            "freq_desc": "ANNUAL",
            "load_time": "2025-11-14 12:00:00.000",
            "end_code": "00",
            "unit_desc": "BU / ACRE",
            "sector_desc": "CROPS",
            "country_code": "9000",
            "group_desc": "FIELD CR

In [6]:
list(data["data"][0].keys())

['watershed_desc',
 'begin_code',
 'country_name',
 'state_fips_code',
 'statisticcat_desc',
 'commodity_desc',
 'source_desc',
 'county_code',
 'week_ending',
 'state_alpha',
 'Value',
 'CV (%)',
 'asd_code',
 'asd_desc',
 'state_ansi',
 'county_ansi',
 'congr_district_code',
 'watershed_code',
 'year',
 'short_desc',
 'freq_desc',
 'load_time',
 'end_code',
 'unit_desc',
 'sector_desc',
 'country_code',
 'group_desc',
 'agg_level_desc',
 'zip_5',
 'county_name',
 'prodn_practice_desc',
 'state_name',
 'domain_desc',
 'domaincat_desc',
 'location_desc',
 'class_desc',
 'reference_period_desc',
 'util_practice_desc',
 'region_desc']

In [74]:
df = pd.DataFrame(data["data"])[["year", "commodity_desc", "Value", "state_alpha", "state_name"]]

In [ ]:
list(data.values()) == [data["data"]] # just checking the json structure

True

In [91]:
type(data["data"])

list

In [92]:
df

,year,commodity_desc,Value,state_alpha,state_name
0,2025,CORN,157.8,OT,OTHER STATES
1,2025,CORN,157.8,OT,OTHER STATES
2,2025,CORN,157.8,OT,OTHER STATES
3,2025,CORN,157.8,OT,OTHER STATES
4,2024,CORN,0,OT,OTHER STATES
...,...,...,...,...,...
823,2021,CORN,209.4,DE,DELAWARE
824,2024,CORN,130.8,DE,DELAWARE
825,2023,CORN,165.7,DE,DELAWARE
826,2022,CORN,126.9,DE,DELAWARE
